In [28]:
import cv2
import numpy as np
import scipy.signal as signal
import scipy.fftpack as fftpack

In [29]:
#convert RBG to YIQ
def rgb2ntsc(src):
    [rows,cols]=src.shape[:2]
    dst=np.zeros((rows,cols,3),dtype=np.float64)
    T = np.array([[0.114, 0.587, 0.298], [-0.321, -0.275, 0.596], [0.311, -0.528, 0.212]])
    for i in range(rows):
        for j in range(cols):
            dst[i, j]=np.dot(T,src[i,j])
    return dst

In [30]:
def ntsc2rbg(src):
    [rows, cols] = src.shape[:2]
    dst=np.zeros((rows,cols,3),dtype=np.float64)
    T = np.array([[1, -1.108, 1.705], [1, -0.272, -0.647], [1, 0.956, 0.620]])
    for i in range(rows):
        for j in range(cols):
            dst[i, j]=np.dot(T,src[i,j])
    return dst

In [31]:
#Build Gaussian Pyramid
def build_gaussian_pyramid(src,level=3):
    s=src.copy()
    pyramid=[s]
    for i in range(level):
        s=cv2.pyrDown(s)
        pyramid.append(s)
    return pyramid

In [32]:
#Build Laplacian Pyramid
def build_laplacian_pyramid(src,levels=3):
    gaussianPyramid = build_gaussian_pyramid(src, levels)
    pyramid=[]
    for i in range(levels,0,-1):
        GE=cv2.pyrUp(gaussianPyramid[i])
        L=cv2.subtract(gaussianPyramid[i-1],GE)
        pyramid.append(L)
    return pyramid

In [33]:
def load_video(video_filename):
    cap=cv2.VideoCapture(video_filename)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_tensor=np.zeros((frame_count,height,width,3),dtype='float')
    x=0
    while cap.isOpened():
        ret,frame=cap.read()
        if ret is True:
            video_tensor[x]=frame
            x+=1
        else:
            break
    return video_tensor,fps

In [34]:
# apply temporal ideal bandpass filter to gaussian video
def temporal_ideal_filter(tensor,low,high,fps,axis=0):
    fft=fftpack.fft(tensor,axis=axis)
    frequencies = fftpack.fftfreq(tensor.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - low)).argmin()
    bound_high = (np.abs(frequencies - high)).argmin()
    fft[:bound_low] = 0
    fft[bound_high:-bound_high] = 0
    fft[-bound_low:] = 0
    iff=fftpack.ifft(fft, axis=axis)
    return np.abs(iff)

In [35]:
# build gaussian pyramid for video
def gaussian_video(video_tensor,levels=3):
    for i in range(0,video_tensor.shape[0]):
        frame=video_tensor[i]
        pyr=build_gaussian_pyramid(frame,level=levels)
        gaussian_frame=pyr[-1]
        if i==0:
            vid_data=np.zeros((video_tensor.shape[0],gaussian_frame.shape[0],gaussian_frame.shape[1],3))
        vid_data[i]=gaussian_frame
    return vid_data

In [36]:
#amplify the video
def amplify_video(gaussian_vid,amplification=50):
    return gaussian_vid*amplification

In [37]:
def reconstract_video(amp_video,origin_video,levels=3):
    final_video=np.zeros(origin_video.shape)
    for i in range(0,amp_video.shape[0]):
        img = amp_video[i]
        for x in range(levels):
            img=cv2.pyrUp(img)
        img=img+origin_video[i]
        final_video[i]=img
    return final_video

In [38]:
def magnify_color(video_name,low,high,levels=3,amplification=50):
    t,f=load_video(video_name)
    print(f)
    gau_video=gaussian_video(t,levels=levels)
    filtered_tensor=temporal_ideal_filter(gau_video,low,high,f)
    amplified_video=amplify_video(filtered_tensor,amplification=amplification)
    final=reconstract_video(amplified_video,t,levels=3)
    print(final)
    return final

In [39]:
final=magnify_color('youness.mp4',1,1.5)

30
[[[[ 75.23245146  76.5696956   85.98158113]
   [ 74.21437932  75.63298529  84.92938686]
   [ 71.52068503  73.12087939  82.10697489]
   ...
   [ 23.74959919   3.34271077  27.00519787]
   [ 23.75934936   3.3506518   27.03433854]
   [ 23.76259941   3.35329881  27.04405209]]

  [[ 77.25273093  78.46892785  88.11207621]
   [ 76.21596291  77.51485571  87.04042803]
   [ 73.47008891  74.9551826   84.16492742]
   ...
   [ 23.67739487   3.30888345  27.12019591]
   [ 23.68655342   3.31663054  27.14924085]
   [ 23.68960626   3.3192129   27.15892249]]

  [[ 82.40447968  83.26000646  93.55647213]
   [ 81.32160891  82.26361564  92.43655006]
   [ 78.44668655  79.58775659  89.42911666]
   ...
   [ 23.47107104   3.22101926  27.38395004]
   [ 23.47865049   3.22828158  27.41266655]
   [ 23.48117698   3.23070235  27.42223872]]

  ...

  [[ 50.27510206  32.48438444  21.53079867]
   [ 49.47805684  32.12043872  21.71301561]
   [ 46.42965899  30.16599228  22.14333399]
   ...
   [ 14.3820466    3.2216287   1

In [40]:
for i in range(0,final.shape[0]):
    cv2.imshow('frame',final[i]/255)
    k=cv2.waitKey(10)
    if k ==ord("q"):
        break
cv2.destroyAllWindows()

In [41]:
cv2.destroyAllWindows()

In [3]:
# def load_video(video_filename):
#     cap=cv2.VideoCapture(0)
#     # frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     # width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     video_tensor=[]
#     x=0
#     while cap.isOpened():
#         ret,frame=cap.read()
        
#         cv2.imshow('frame',frame)
#         video_tensor.append(frame)
#         x+=1
        
#     video_tensor=np.array(video_tensor)
#     return video_tensor,fps
# load_video('youness.mp4')

KeyboardInterrupt: 

In [1]:
cap = cv2.VideoCapture(0)
vid_data=[]
while cap.isOpened():
    ret,frame=cap.read()
    if ret is True:
        cv2.imshow('frame',frame)
        vid_data.append(frame)
        k=cv2.waitKey(10)
        if k ==ord("q"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
print(np.array(vid_data))

NameError: name 'cv2' is not defined

In [155]:
import time
cap = cv2.VideoCapture(0)
fps = int(cap.get(cv2.CAP_PROP_FPS))
vid_data=[]
curr_time=time.time()
while cap.isOpened():
    ret,frame=cap.read()
    if time.time()-curr_time>10:
        break
    frame=cv2.flip(frame,1)
    if ret is True:
        cv2.imshow('frame',frame)
        vid_data.append(frame)
        k=cv2.waitKey(10)
        if k ==ord("q"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
vid_data=np.array(vid_data)

In [156]:
def magnify_color(video_name,low,high,levels=3,amplification=50):
    # t,f=load_video(video_name)
    t=vid_data
    f=fps
    print(f)
    gau_video=gaussian_video(t,levels=levels)
    filtered_tensor=temporal_ideal_filter(gau_video,low,high,f)
    amplified_video=amplify_video(filtered_tensor,amplification=amplification)
    final=reconstract_video(amplified_video,t,levels=levels)
    print(final)
    return final

In [157]:
final=magnify_color('youness.mp4',1,1.5,4,50)
print(final,cv2.convertScaleAbs(final[0]))

30
[[[[260.91017598 241.68558606 239.66065199]
   [261.89450968 242.70923469 241.69677178]
   [258.8532167  240.77414281 238.80168629]
   ...
   [230.98035153 200.22891989 267.03849507]
   [230.98436074 200.22492492 267.06052585]
   [230.98569714 200.22359326 267.06786945]]

  [[260.92617146 237.65477634 234.60199996]
   [261.91034272 238.67853664 235.63821529]
   [261.86859901 240.74375536 237.74340841]
   ...
   [232.01758173 200.23974543 269.97990116]
   [235.02157206 203.23575848 272.00190507]
   [232.02290217 200.23442949 270.00923971]]

  [[261.96981694 239.56876684 235.43697294]
   [261.95353208 238.5928316  235.47347185]
   [261.91052235 238.65889744 236.57949041]
   ...
   [231.12314167 201.27152993 272.808056  ]
   [230.12707729 200.26756572 271.82998062]
   [228.12838916 198.26624431 269.83728883]]

  ...

  [[189.22581409 169.99943755 166.15630322]
   [189.26168952 169.95252999 166.13654508]
   [190.35989106 170.82079613 167.0833473 ]
   ...
   [178.94612532 183.23412324 19

In [158]:
for i in range(0,final.shape[0]):
    cv2.imshow('frame',cv2.convertScaleAbs(final[i]))
    k=cv2.waitKey(10)
    if k ==ord("q"):
        break
cv2.destroyAllWindows()

In [150]:
cv2.destroyAllWindows()

In [151]:
def save_video(video_tensor):
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    [height,width]=video_tensor[0].shape[0:2]
    writer = cv2.VideoWriter("out.avi", fourcc, 30, (width, height), 1)
    for i in range(0,video_tensor.shape[0]):
        writer.write(cv2.convertScaleAbs(video_tensor[i]))
        cv2.imshow('frame',cv2.convertScaleAbs(video_tensor[i]))
        k=cv2.waitKey(10)
        if k ==ord("q"):
            break
    writer.release()
save_video(final)
cv2.destroyAllWindows()